In [ ]:
# Install TensorFlow
# !pip install -q tensorflow-gpu==2.0.0-beta1

try:
  %tensorflow_version 2.x  # Colab only.
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)
     |████████████████████████████████| 348.9MB 52kB/s 
     |████████████████████████████████| 3.1MB 49.1MB/s 
     |████████████████████████████████| 501kB 54.6MB/s 
2.0.0-beta1
# Load in the data
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print("x_train.shape:", x_train.shape)
Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
11493376/11490434 [==============================] - 0s 0us/step
x_train.shape: (60000, 28, 28)
# Build the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Train the model
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)
WARNING: Logging before flag parsing goes to stderr.
W0718 16:30:06.976897 139639418169216 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 111us/sample - loss: 0.2949 - accuracy: 0.9134 - val_loss: 0.1339 - val_accuracy: 0.9594
Epoch 2/10
60000/60000 [==============================] - 6s 93us/sample - loss: 0.1452 - accuracy: 0.9564 - val_loss: 0.1063 - val_accuracy: 0.9681
Epoch 3/10
60000/60000 [==============================] - 6s 95us/sample - loss: 0.1100 - accuracy: 0.9664 - val_loss: 0.0879 - val_accuracy: 0.9722
Epoch 4/10
60000/60000 [==============================] - 6s 94us/sample - loss: 0.0902 - accuracy: 0.9721 - val_loss: 0.0737 - val_accuracy: 0.9766
Epoch 5/10
60000/60000 [==============================] - 6s 93us/sample - loss: 0.0765 - accuracy: 0.9758 - val_loss: 0.0747 - val_accuracy: 0.9748
Epoch 6/10
60000/60000 [==============================] - 6s 94us/sample - loss: 0.0672 - accuracy: 0.9789 - val_loss: 0.0699 - val_accuracy: 0.9772
Epoch 7/10
60000/60000 [==============================] - 6s 93us/sample - loss: 0.0586 - accuracy: 0.9820 - val_loss: 0.0729 - val_accuracy: 0.9781
Epoch 8/10
60000/60000 [==============================] - 6s 94us/sample - loss: 0.0544 - accuracy: 0.9815 - val_loss: 0.0697 - val_accuracy: 0.9780
Epoch 9/10
60000/60000 [==============================] - 6s 93us/sample - loss: 0.0505 - accuracy: 0.9834 - val_loss: 0.0690 - val_accuracy: 0.9789
Epoch 10/10
60000/60000 [==============================] - 6s 93us/sample - loss: 0.0447 - accuracy: 0.9850 - val_loss: 0.0692 - val_accuracy: 0.9806
# Plot loss per iteration
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
<matplotlib.legend.Legend at 0x7effe00f2ac8>

# Plot accuracy per iteration
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()
<matplotlib.legend.Legend at 0x7effe0092518>

# Evaluate the model
print(model.evaluate(x_test, y_test))
10000/10000 [==============================] - 1s 55us/sample - loss: 0.0692 - accuracy: 0.9806
[0.06924617666350968, 0.9806]
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """
  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
  else:
      print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()


p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))

# Do these results make sense?
# It's easy to confuse 9 <--> 4, 9 <--> 7, 2 <--> 7, etc. 
Confusion matrix, without normalization
[[ 972    0    1    1    0    0    2    1    2    1]
 [   0 1126    2    2    0    0    2    0    3    0]
 [   2    1 1015    3    1    0    1    4    4    1]
 [   1    0    1  996    0    2    0    3    3    4]
 [   1    1    2    1  959    0    5    3    1    9]
 [   2    1    0   12    1  868    2    2    3    1]
 [   5    3    1    1    1    3  939    0    5    0]
 [   2   10   12    3    0    0    0  994    3    4]
 [   6    0    3    2    3    1    0    2  953    4]
 [   2    2    0    3   12    3    0    3    0  984]]

# Show some misclassified examples
misclassified_idx = np.where(p_test != y_test)[0]
i = np.random.choice(misclassified_idx)
plt.imshow(x_test[i], cmap='gray')
plt.title("True label: %s Predicted: %s" % (y_test[i], p_test[i]));



In [ ]:
# Install TensorFlow
# !pip install -q tensorflow-gpu==2.0.0-beta1

try:
  %tensorflow_version 2.x  # Colab only.
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)
2.0.0-beta1
# Other imports
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# Make the dataset
N = 1000
X = np.random.random((N, 2)) * 6 - 3 # uniformly distributed between (-3, +3)
Y = np.cos(2*X[:,0]) + np.cos(3*X[:,1])
This implements the function:

y
=
cos
(
2
x
1
)
+
c
o
s
(
3
x
2
)
# Plot it
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:,0], X[:,1], Y)
# plt.show()
<mpl_toolkits.mplot3d.art3d.Path3DCollection at 0x7f2e953549e8>

# Build the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, input_shape=(2,), activation='relu'),
  tf.keras.layers.Dense(1)
])
# Compile and fit
opt = tf.keras.optimizers.Adam(0.01)
model.compile(optimizer=opt, loss='mse')
r = model.fit(X, Y, epochs=100)
Train on 1000 samples
Epoch 1/100
1000/1000 [==============================] - 0s 493us/sample - loss: 0.9276
Epoch 2/100
1000/1000 [==============================] - 0s 58us/sample - loss: 0.9060
Epoch 3/100
1000/1000 [==============================] - 0s 68us/sample - loss: 0.8808
Epoch 4/100
1000/1000 [==============================] - 0s 54us/sample - loss: 0.8457
Epoch 5/100
1000/1000 [==============================] - 0s 57us/sample - loss: 0.8115
Epoch 6/100
1000/1000 [==============================] - 0s 59us/sample - loss: 0.7682
Epoch 7/100
1000/1000 [==============================] - 0s 59us/sample - loss: 0.6904
Epoch 8/100
1000/1000 [==============================] - 0s 58us/sample - loss: 0.6319
Epoch 9/100
1000/1000 [==============================] - 0s 51us/sample - loss: 0.5543
Epoch 10/100
1000/1000 [==============================] - 0s 58us/sample - loss: 0.5207
Epoch 11/100
1000/1000 [==============================] - 0s 54us/sample - loss: 0.4973
Epoch 12/100
1000/1000 [==============================] - 0s 52us/sample - loss: 0.4889
Epoch 13/100
1000/1000 [==============================] - 0s 58us/sample - loss: 0.4715
Epoch 14/100
1000/1000 [==============================] - 0s 55us/sample - loss: 0.4714
Epoch 15/100
1000/1000 [==============================] - 0s 53us/sample - loss: 0.4387
Epoch 16/100
1000/1000 [==============================] - 0s 58us/sample - loss: 0.4519
Epoch 17/100
1000/1000 [==============================] - 0s 53us/sample - loss: 0.4498
Epoch 18/100
1000/1000 [==============================] - 0s 53us/sample - loss: 0.4180
Epoch 19/100
1000/1000 [==============================] - 0s 53us/sample - loss: 0.4163
Epoch 20/100
1000/1000 [==============================] - 0s 54us/sample - loss: 0.4128
Epoch 21/100
1000/1000 [==============================] - 0s 67us/sample - loss: 0.4110
Epoch 22/100
1000/1000 [==============================] - 0s 62us/sample - loss: 0.3984
Epoch 23/100
1000/1000 [==============================] - 0s 58us/sample - loss: 0.3825
Epoch 24/100
1000/1000 [==============================] - 0s 52us/sample - loss: 0.3734
Epoch 25/100
1000/1000 [==============================] - 0s 54us/sample - loss: 0.4398
Epoch 26/100
1000/1000 [==============================] - 0s 57us/sample - loss: 0.3796
Epoch 27/100
1000/1000 [==============================] - 0s 53us/sample - loss: 0.3603
Epoch 28/100
1000/1000 [==============================] - 0s 54us/sample - loss: 0.3120
Epoch 29/100
1000/1000 [==============================] - 0s 58us/sample - loss: 0.2797
Epoch 30/100
1000/1000 [==============================] - 0s 53us/sample - loss: 0.2815
Epoch 31/100
1000/1000 [==============================] - 0s 54us/sample - loss: 0.2600
Epoch 32/100
1000/1000 [==============================] - 0s 54us/sample - loss: 0.2221
Epoch 33/100
1000/1000 [==============================] - 0s 52us/sample - loss: 0.2036
Epoch 34/100
1000/1000 [==============================] - 0s 59us/sample - loss: 0.1905
Epoch 35/100
1000/1000 [==============================] - 0s 54us/sample - loss: 0.1502
Epoch 36/100
1000/1000 [==============================] - 0s 51us/sample - loss: 0.1368
Epoch 37/100
1000/1000 [==============================] - 0s 51us/sample - loss: 0.1300
Epoch 38/100
1000/1000 [==============================] - 0s 52us/sample - loss: 0.1043
Epoch 39/100
1000/1000 [==============================] - 0s 66us/sample - loss: 0.1022
Epoch 40/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0823
Epoch 41/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0787
Epoch 42/100
1000/1000 [==============================] - 0s 51us/sample - loss: 0.0593
Epoch 43/100
1000/1000 [==============================] - 0s 53us/sample - loss: 0.0528
Epoch 44/100
1000/1000 [==============================] - 0s 48us/sample - loss: 0.0616
Epoch 45/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0351
Epoch 46/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0273
Epoch 47/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0288
Epoch 48/100
1000/1000 [==============================] - 0s 51us/sample - loss: 0.0281
Epoch 49/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0362
Epoch 50/100
1000/1000 [==============================] - 0s 48us/sample - loss: 0.0585
Epoch 51/100
1000/1000 [==============================] - 0s 52us/sample - loss: 0.0842
Epoch 52/100
1000/1000 [==============================] - 0s 54us/sample - loss: 0.0780
Epoch 53/100
1000/1000 [==============================] - 0s 53us/sample - loss: 0.0700
Epoch 54/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0232
Epoch 55/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0149
Epoch 56/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0159
Epoch 57/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0140
Epoch 58/100
1000/1000 [==============================] - 0s 52us/sample - loss: 0.0108
Epoch 59/100
1000/1000 [==============================] - 0s 66us/sample - loss: 0.0088
Epoch 60/100
1000/1000 [==============================] - 0s 52us/sample - loss: 0.0077
Epoch 61/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0132
Epoch 62/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0108
Epoch 63/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0105
Epoch 64/100
1000/1000 [==============================] - 0s 53us/sample - loss: 0.0077
Epoch 65/100
1000/1000 [==============================] - 0s 51us/sample - loss: 0.0084
Epoch 66/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0101
Epoch 67/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0093
Epoch 68/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0082
Epoch 69/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0076
Epoch 70/100
1000/1000 [==============================] - 0s 58us/sample - loss: 0.0145
Epoch 71/100
1000/1000 [==============================] - 0s 47us/sample - loss: 0.0127
Epoch 72/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0101
Epoch 73/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0058
Epoch 74/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0051
Epoch 75/100
1000/1000 [==============================] - 0s 53us/sample - loss: 0.0053
Epoch 76/100
1000/1000 [==============================] - 0s 52us/sample - loss: 0.0073
Epoch 77/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0065
Epoch 78/100
1000/1000 [==============================] - 0s 53us/sample - loss: 0.0066
Epoch 79/100
1000/1000 [==============================] - 0s 60us/sample - loss: 0.0202
Epoch 80/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0130
Epoch 81/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0093
Epoch 82/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0149
Epoch 83/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0147
Epoch 84/100
1000/1000 [==============================] - 0s 46us/sample - loss: 0.0085
Epoch 85/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0048
Epoch 86/100
1000/1000 [==============================] - 0s 51us/sample - loss: 0.0041
Epoch 87/100
1000/1000 [==============================] - 0s 48us/sample - loss: 0.0041
Epoch 88/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0106
Epoch 89/100
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0112
Epoch 90/100
1000/1000 [==============================] - 0s 47us/sample - loss: 0.0051
Epoch 91/100
1000/1000 [==============================] - 0s 48us/sample - loss: 0.0121
Epoch 92/100
1000/1000 [==============================] - 0s 52us/sample - loss: 0.0384
Epoch 93/100
1000/1000 [==============================] - 0s 51us/sample - loss: 0.0463
Epoch 94/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0190
Epoch 95/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0340
Epoch 96/100
1000/1000 [==============================] - 0s 52us/sample - loss: 0.0133
Epoch 97/100
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0062
Epoch 98/100
1000/1000 [==============================] - 0s 67us/sample - loss: 0.0043
Epoch 99/100
1000/1000 [==============================] - 0s 67us/sample - loss: 0.0050
Epoch 100/100
1000/1000 [==============================] - 0s 51us/sample - loss: 0.0045
# Plot the loss
plt.plot(r.history['loss'], label='loss')
[<matplotlib.lines.Line2D at 0x7f2e95406a58>]

# Plot the prediction surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:,0], X[:,1], Y)

# surface plot
line = np.linspace(-3, 3, 50)
xx, yy = np.meshgrid(line, line)
Xgrid = np.vstack((xx.flatten(), yy.flatten())).T
Yhat = model.predict(Xgrid).flatten()
ax.plot_trisurf(Xgrid[:,0], Xgrid[:,1], Yhat, linewidth=0.2, antialiased=True)
plt.show()

# Can it extrapolate?
# Plot the prediction surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:,0], X[:,1], Y)

# surface plot
line = np.linspace(-5, 5, 50)
xx, yy = np.meshgrid(line, line)
Xgrid = np.vstack((xx.flatten(), yy.flatten())).T
Yhat = model.predict(Xgrid).flatten()
ax.plot_trisurf(Xgrid[:,0], Xgrid[:,1], Yhat, linewidth=0.2, antialiased=True)
plt.show()

sss